# Assignment: 1 - Camera Calibration 

### Importing necessary libraries

In [45]:
import numpy as np
import pandas as pd
from scipy import linalg

### Normalization

In [46]:
def Normalization(nd, x):
    '''
    Normalization of coordinates (centroid to the origin and mean distance of sqrt(2 or 3).

    Input
    -----
    nd: number of dimensions, 3 here
    x: the data to be normalized (directions at different columns and points at rows)
    Output
    ------
    Tr: the transformation matrix (translation plus scaling)
    x: the transformed data
    '''

    #CONVERTING TO NUMPY ARRAY
    x = np.asarray(x)

    #CALCULATING MEAN AND STANDAR DEVIATION
    m, s = np.mean(x, 0), np.std(x)

    print("mean = ", m, "std = ", s)

    #NORMALIZATION MATRIX FOR WORLD POINTS(3D) AND IMAGE-PIXEL POINTS(2D)
    if nd == 2:
        Tr = np.array([[s, 0, m[0]], [0, s, m[1]], [0, 0, 1]])
    else:
        Tr = np.array([[s, 0, 0, m[0]], [0, s, 0, m[1]], [0, 0, s, m[2]], [0, 0, 0, 1]])
        
    Tr = np.linalg.inv(Tr)
    x = np.dot( Tr, np.concatenate( (x.T, np.ones((1,x.shape[0]))) ) )
    x = x[0:nd, :].T

    return Tr, x

### DLT Camera Calibration

In [47]:
def DLTcalib(nd, xyz, img_pt_):
    '''
    Camera calibration by DLT using known object points and their image points.

    Input
    -----
    nd: dimensions of the object space, 3 here.
    xyz: coordinates in the object 3D space.
    img_pt_: coordinates in the image 2D space.

    The coordinates (x,y,z and u,v) are given as columns and the different points as rows.

    There must be at least 6 calibration points for the 3D DLT.

    Output
    ------
     L: array of 11 parameters of the calibration matrix.
     err: error of the DLT (mean residual of the DLT transformation in units of camera coordinates).
    '''
    if (nd != 3):
        raise ValueError('%dD DLT unsupported.' %(nd))
    
    # Converting all variables to numpy array
    xyz = np.asarray(xyz)
    img_pt_ = np.asarray(img_pt_)

    n = xyz.shape[0]

    print("\n\nNUMBER OF POINTS: {}\n\n".format(n))
    # Validating the parameters:
    if img_pt_.shape[0] != n:
        raise ValueError('Object (%d points) and image (%d points) have different number of points.' %(n, img_pt_.shape[0]))

    if (xyz.shape[1] != 3):
        raise ValueError('Incorrect number of coordinates (%d) for %dD DLT (it should be %d).' %(xyz.shape[1],nd,nd))

    if (n < 6):
        raise ValueError('%dD DLT requires at least %d calibration points. Only %d points were entered.' %(nd, 2*nd, n))
        
    # Normalize the data to improve the DLT quality (DLT is dependent of the system of coordinates).
    # This is relevant when there is a considerable perspective distortion.
    # Normalization: mean position at origin and mean distance equals to 1 at each direction.
    Txyz, xyzn = Normalization(nd, xyz)
    Timg_pt_, img_pt_n = Normalization(2, img_pt_)

    A = []

    for i in range(n):
        x, y, z = xyzn[i, 0], xyzn[i, 1], xyzn[i, 2]
        u, v = img_pt_n[i, 0], img_pt_n[i, 1]
        A.append( [x, y, z, 1, 0, 0, 0, 0, -u * x, -u * y, -u * z, -u] )
        A.append( [0, 0, 0, 0, x, y, z, 1, -v * x, -v * y, -v * z, -v] )

    # Convert A to array
    A = np.asarray(A) 

    # Find the 11 parameters:
    U, S, V = np.linalg.svd(A)

    # The parameters are in the last line of Vh and normalize them
    L = V[-1, :] / V[-1, -1]

    # Camera projection matrix
    H = L.reshape(3, nd + 1)


    # Denormalization
    H = np.dot( np.dot( np.linalg.pinv(Timg_pt_), H ), Txyz )
    H = H / H[-1, -1]
    L = H.flatten()

    # Mean error of the DLT (mean residual of the DLT transformation in units of camera coordinates):
    img_pt_2 = np.dot( H, np.concatenate( (xyz.T, np.ones((1, xyz.shape[0]))) ) ) 
    img_pt_2 = img_pt_2 / img_pt_2[2, :] 

    # Mean distance:
    projected_pts = img_pt_2.T
    actual_pixels = img_pt_

    # Printing teh actual and the estimated points
    print('\n\nACTUAL PTS   |             ESTIMATED PTS')
    for i in range(len(img_pt_2.T)):
        print("{}   <---->  {}".format(actual_pixels[i], projected_pts[i]))

    err = np.sqrt( np.mean(np.sum( (img_pt_2[0:2, :].T - img_pt_)**2, 1)) ) 

    return L, err

In [48]:
def get_intrest_points():

    # WORLD CORDINATES
    data = pd.read_excel("dataset.xlsx")
    x = np.array(data['X'])
    y = np.array(data['Y'])
    z = np.array(data['Z'])

    # IMAGE PIXEL COORDINATES
    u = np.array(data['x'])
    v= np.array(data['y'])

    # LIST OF WORLD PTS AND CORRESPONDING PIXEL PTS
    world_pts = []
    pixel_pts = []

    for i in range(len(x)):
        world_pts.append([x[i], y[i], z[i]])
        pixel_pts.append([u[i], v[i]])

    print("DATA FRAME")
    print(data)

    from sklearn.utils import shuffle
    world_pts, pixel_pts = shuffle(world_pts, pixel_pts, random_state = 0)

    print("\n\nWORLD points")
    print(world_pts)
    return world_pts, pixel_pts
   

### RQ Decomposition 

In [49]:
def RQ_decomposition(P):
    M = P[0:3,0:3]
    print("\nM:\n",M)
    K, R = linalg.rq(M)
    T = np.diag(np.sign(np.diag(K)))
    print(T)

    K = np.dot(K, T)
    R = np.dot(T, R)
    C = np.dot(linalg.inv(-M), P[:, 3])
    return K, R, C


In [50]:
def camera_param(P):
    print("\nP:\n",P)

    K, R, C = RQ_decomposition(P)

    print("\n\n K MATRIX: ")
    print(K)
    print("\n\n R MATRIX: ")
    print(R)
    print("\n\n Camera center: ")
    print(C)

    print("\n\n Normalized camera matrix: ")
    print(K/K[2][2])


In [51]:
#camera_param(P)

In [52]:

xyz, img_pt_ = get_intrest_points()
nd = 3
P, err = DLTcalib(nd, xyz, img_pt_)

P = P.reshape(3,4)

[20.   0.   0.  20.  20.   0.   0.   0.  12.5  5.  10.  10.   0.   0.
 15.  15.  25.   0. ]
[ 0.  20.   0.  20.   0.  20.  15.   7.5 17.5  0.  10.   0.  10.   0.
 10.   0.   0.  25. ]
[ 0  0 20  0 20 20 10 15  0  5  0 10 10  0  0 10 25 25]

[708 213 455 464 727 182 279 373 396 516 463 576 343 463 529 642 829  76]
[ 991  998  597 1192  658  665  810  699 1082  828  998  779  784  874
 1038  805  576  586]
DATA FRAME
       X     Y   Z    x     y  Unnamed: 5
0   20.0   0.0   0  708   991           1
1    0.0  20.0   0  213   998           2
2    0.0   0.0  20  455   597           3
3   20.0  20.0   0  464  1192           4
4   20.0   0.0  20  727   658           5
5    0.0  20.0  20  182   665           6
6    0.0  15.0  10  279   810           7
7    0.0   7.5  15  373   699           8
8   12.5  17.5   0  396  1082           9
9    5.0   0.0   5  516   828          10
10  10.0  10.0   0  463   998          11
11  10.0   0.0  10  576   779          12
12   0.0  10.0  10  343   784      

### Projection matrix (P)

In [53]:
print('Matrix')
print(P)

Matrix
[[ 5.69722141e+00 -1.44186467e+01 -2.04509681e+00  4.62512584e+02]
 [-3.37417373e+00 -3.24421920e+00 -1.60998815e+01  8.72718072e+02]
 [-9.38707861e-03 -9.59506372e-03 -3.83990800e-03  1.00000000e+00]]


### RMSE

In [54]:
print('\nError')
print(err)


Error
2.0684280927800307


### Camera Parameters

In [55]:
camera_param(P.reshape(3,4))


P:
 [[ 5.69722141e+00 -1.44186467e+01 -2.04509681e+00  4.62512584e+02]
 [-3.37417373e+00 -3.24421920e+00 -1.60998815e+01  8.72718072e+02]
 [-9.38707861e-03 -9.59506372e-03 -3.83990800e-03  1.00000000e+00]]

M:
 [[ 5.69722141e+00 -1.44186467e+01 -2.04509681e+00]
 [-3.37417373e+00 -3.24421920e+00 -1.60998815e+01]
 [-9.38707861e-03 -9.59506372e-03 -3.83990800e-03]]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]


 K MATRIX: 
[[1.41572193e+01 8.45505830e-02 6.64109330e+00]
 [0.00000000e+00 1.41929350e+01 8.92618558e+00]
 [0.00000000e+00 0.00000000e+00 1.39616398e-02]]


 R MATRIX: 
[[ 0.71671523 -0.69729852 -0.00969774]
 [ 0.1851152   0.2036404  -0.96138595]
 [-0.67234786 -0.68724475 -0.27503274]]


 Camera center: 
[45.8491945  45.15883126 35.49776286]
Normalized camera matrix: 
[[1.01400834e+03 6.05592067e+00 4.75667142e+02]
 [0.00000000e+00 1.01656648e+03 6.39336475e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
